In [4]:
# Saves models to tmp folder because the persistent folder doesn't have enough space

#!rm -rf /home/studio-lab-user/.cache/huggingface
import os
os.makedirs('/tmp/models/huggingface', exist_ok=True)
os.makedirs('/home/studio-lab-user/.cache', exist_ok=True)
!ln -s /tmp/models/huggingface /home/studio-lab-user/.cache

In [ ]:
# Set variables and install

BASEPATH = '/home/studio-lab-user/sagemaker-studiolab-notebooks'

%cd $BASEPATH
!git clone https://github.com/hotshotco/Hotshot-XL
HOTSHOT_PATH = BASEPATH + '/Hotshot-XL'
%cd $HOTSHOT_PATH
!pip install -r requirements.txt

import os
os.makedirs('lora', exist_ok=True)
os.makedirs('input', exist_ok=True)
os.makedirs('output', exist_ok=True)


In [ ]:
#@title Download a LoRA
url = 'https://civitai.com/api/download/models/177248'  #@param {type:"string"}
lora_file_name = 'vangogh.safetensors' #@param {type:"string"}

%cd $HOTSHOT_PATH
%cd lora

import requests

r = requests.get(url, allow_redirects=True)
with open(lora_file_name, 'wb') as f:
    f.write(r.content)

print('Downloaded ' + lora_file_name)

In [ ]:
# Download a GIF
url='https://media2.giphy.com/media/v1.Y2lkPTc5MGI3NjExZGFiY2M1bnl0bjVrNjdxYWViZTZsZHZwOXNncmNjbGhuam9lM3F0ZSZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/0VKbVY3N0VW0ZegLbe/giphy.gif' #@param {type:"string"}
gif_name = 'bunny.gif' #@param {type:"string"}

%cd $HOTSHOT_PATH
%cd input

import requests
from PIL import Image
from io import BytesIO

r = requests.get(url, allow_redirects=True)
with open(gif_name, 'wb') as f:
    f.write(r.content)

# Show the frame count
im = Image.open(gif_name)
print(f'{gif_name} has {im.n_frames} frames')

In [26]:
%cd $HOTSHOT_PATH

# Parameters

batch_size = 2 # @param {type:"slider", min:1, max:50, step:1}
positive_prompt = 'van gogh painting of a bunny rabbit running, side view, black eyes, octane render, grass, flowers, dramatic lighting, pixar style, nature, enchanting, wonderfully colored, vibrant, cute'
lora_file_name = 'vangogh.safetensors' #@param {type:"string"}
gif_name = 'bunny.gif' #@param {type:"string"}
width = 608 #@param {type:"integer"}
height = 416 #@param {type:"integer"}
controlnet_conditioning_scale = 0.7 #@param {type:"number"}

negative_prompt = 'blurry, fog' #@param {type:"string"}
seed = -1 #@param {type:"integer"}

upscale = "0" #@param [0, 2, 4]
video_frames = 8 #@param {type:"integer"}
video_duration = 1000 #@param {type:"integer"}
steps = "30" #@param {type:"string"}

file_prefix = '1015bun_' #@param {type:"string"}
file_type = '.mp4' #@param [".mp4", ".gif"]
scheduler='EulerAncestralDiscreteScheduler' #@param ["EulerAncestralDiscreteScheduler", "EulerDiscreteScheduler"]

# 
# Gif to video

control_type = 'depth' #@param ["depth", "canny"]
controlnet_conditioning_scale = 0.8 #@param {type:"number"}
control_guidance_start = 0.0 #@param {type:"number"}
control_guidance_end = 1.0 #@param {type:"number"}

# Resolutions quick reference

# | Width x Height | Ratio| Orientation |
# | --- | --- | --- |
# | 512 x 512 | 1 : 1 | Square |
# | 608 x 416 | 19 : 13 | Horizontal
# | 416 x 608 | 13 : 19 | Vertical
# | 672 x 384 | 7 : 4 | Horizontal
# | 384 x 672 | 4 : 7 | Vertical
# | 768 x 320 | 12 : 5 | Horizontal
# | 320 x 768 | 5 : 12 | Vertical

# No need to edit below this point

import datetime, random, os
from IPython.display import display, Image, HTML
from base64 import b64encode

videos = []

if seed == None or seed == -1:
  seed = random.randint(0, 18446744073709519871)

for i in range(batch_size):
  file_name=f'{file_prefix}{datetime.datetime.now().strftime("%Y%m%d-%H%M%S")}{file_type}'
  data_name=f'{file_prefix}{datetime.datetime.now().strftime("%Y%m%d-%H%M%S")}.txt'
  # Create the command
  cmd = f'''python inference.py \
   --prompt="{positive_prompt}" \
   --negative_prompt="{negative_prompt}" \
   --steps={steps} \
   --seed={seed} \
   --width={width} \
   --height={height} \
   --video_length={video_frames} \
   --video_duration={video_duration} \
   --scheduler="{scheduler}" '''
  if upscale == '2' or upscale == '4':
    cmd += f' --upscale={upscale}  '
  if not lora_file_name == '':
    print('using lora')
    cmd += f' --lora="lora/{lora_file_name}"  '
  if not gif_name == '':
    cmd += f''' --gif="input/{gif_name}" \
      --control_type={control_type} \
      --controlnet_conditioning_scale={controlnet_conditioning_scale} \
      --control_guidance_start={control_guidance_start} \
      --control_guidance_end={control_guidance_end} '''

  cmd += f' --output="output/{file_name}"'

  # Execute command
  print(cmd)
  get_ipython().system(cmd);

  # Write command to file
  f = open('output/'+data_name, 'w')
  f.write(cmd)
  f.close()

  seed = random.randint(0, 18446744073709519871)
  videos.append('output/' + file_name)

# Show the results
print(str(videos))
result = ''
if file_type == '.gif':
  for v in videos:
    display(Image(filename=v, retina=True))
else:
  for v in videos:
      mp4 = open(v,'rb').read()
      data_url = 'data:video/mp4;base64,' + b64encode(mp4).decode()
      result += ('''
      <video width=400 controls loop>
        <source src="%s" type="video/mp4">
      </video>
      ''' % data_url)
HTML(result)

/home/studio-lab-user/sagemaker-studiolab-notebooks/Hotshot-XL
using lora
python inference.py    --prompt="van gogh painting of a bunny rabbit running, side view, black eyes, octane render, grass, flowers, dramatic lighting, pixar style, nature, enchanting, wonderfully colored, vibrant, cute"    --negative_prompt="blurry, fog"    --steps=30    --seed=2913947611431714917    --width=608    --height=416    --video_length=8    --video_duration=1000    --scheduler="EulerAncestralDiscreteScheduler"  --lora="lora/vangogh.safetensors"   --gif="input/bunFF.GIF"       --control_type=depth       --controlnet_conditioning_scale=0.8       --control_guidance_start=0.0       --control_guidance_end=1.0  --output="output/1015bun_20231015-191837.mp4"
100%|█████████████████████████████████████████████| 8/8 [00:01<00:00,  4.60it/s]
using lora
python inference.py    --prompt="van gogh painting of a bunny rabbit running, side view, black eyes, octane render, grass, flowers, dramatic lighting, pixar style, n

In [ ]:
# zip the output folder for easy download
%cd $HOTSHOT_PATH
!tar -czf output.tar.gz output